# LangChain Expression Language


In [1]:

from dotenv import load_dotenv
load_dotenv()

True

Old way



In [2]:

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [3]:
llm = ChatOpenAI()


/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
prompt = ChatPromptTemplate.from_template("tell me about the nutritional value of {input}")

In [5]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="Pizza")

'Pizza can vary greatly in nutritional value depending on the toppings and crust used. However, a typical slice of cheese pizza (1/8 of a 14-inch pizza) has approximately 285 calories, 10 grams of fat, 36 grams of carbohydrates, and 12 grams of protein.\n\nPizza can be a good source of several essential nutrients, including calcium from the cheese, protein from the cheese and crust, and vitamins and minerals from the tomato sauce and toppings. However, it can also be high in sodium, saturated fat, and calories, especially if it is loaded with high-fat meats and extra cheese.\n\nTo make pizza more nutritious, consider opting for a whole wheat crust, adding plenty of vegetables as toppings, and choosing lean proteins like grilled chicken or shrimp. Additionally, limiting portion sizes and enjoying pizza in moderation can help prevent it from becoming a high-calorie, high-fat meal.'

New way

In [6]:
chain = prompt | llm
chain.invoke({"input": "Spaghetti"})

AIMessage(content='Spaghetti is a popular pasta dish made from durum wheat semolina flour and water. It is a good source of carbohydrates, providing energy to fuel the body. A one-cup serving of cooked spaghetti contains approximately 200 calories, 40 grams of carbohydrates, 7 grams of protein, and less than 1 gram of fat.\n\nSpaghetti also contains essential vitamins and minerals such as iron, magnesium, and B vitamins. Iron is important for the production of red blood cells and magnesium is essential for muscle and nerve function. B vitamins play a role in energy metabolism and overall health.\n\nSpaghetti is typically served with a tomato-based sauce, which adds additional nutrients such as vitamin C and lycopene. Lycopene is a powerful antioxidant that may help reduce the risk of certain chronic diseases.\n\nOverall, spaghetti can be a nutritious and satisfying meal when paired with a balanced sauce and other healthy ingredients such as vegetables and lean protein. It is important 

In [7]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'input': {'title': 'Input', 'type': 'string'}}}

In [8]:
chain.input_schema.schema()


{'title': 'PromptInput',
 'type': 'object',
 'properties': {'input': {'title': 'Input', 'type': 'string'}}}

# First pipe

In [9]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [10]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "Lasagna"})

"Lasagna is a classic Italian dish made with layers of pasta, cheese, meat, and tomato sauce. It is a rich and satisfying meal that provides a good source of essential nutrients.\n\nThe nutritional value of lasagna can vary depending on the specific ingredients used in the recipe. However, a typical serving of lasagna (about 1 cup) can provide the following nutrients:\n\n- Protein: Lasagna is a good source of protein, thanks to the meat and cheese layers. Protein is important for building and repairing tissues in the body.\n- Carbohydrates: The pasta in lasagna provides a source of carbohydrates, which are the body's main source of energy.\n- Fat: Lasagna can be high in fat, especially if it contains a lot of cheese or meat. However, some fats, such as those found in olive oil, can be beneficial for heart health.\n- Fiber: Depending on the type of pasta and vegetables used, lasagna can also be a good source of dietary fiber, which is important for digestive health.\n- Vitamins and mine

In [11]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [12]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()


In [13]:
chain.invoke({"input": "pizzas"})

'1. Why did the pizza maker go to therapy? Because he kneaded it.'

# OPENAI Functions

In [14]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [15]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt
    | llm.bind(function_call={"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [16]:
chain.invoke(input={"input": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

# Working with vectorstores

In [19]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.",
                                 "Cats have retractable claws.",
                                 "A group of cats is called a clowder.",
                                 "Cats can rotate their ears 180 degrees.",
                                 "The world's oldest cat lived to be 38 years old."],
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [21]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
chain.invoke("how old is the oldest cat?")

'The oldest cat lived to be 38 years old.'

In [23]:
from operator import itemgetter

In [24]:
question = {"bla": "test", "x": "hi"}
itemgetter("bla")

operator.itemgetter('bla')

In [25]:
get_bla = itemgetter("bla")
get_bla(question)

'test'

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | llm | StrOutputParser()

# chain = {
#     "context": (lambda x: x["question"]) | retriever,
#     "question": (lambda x: x["question"]),
#     "language": (lambda x: x["language"])
# } | prompt | llm | StrOutputParser()

In [27]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})


'38 Jahre alt.'

In [28]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01


In [29]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [30]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [31]:
chain = prompt | llm | StrOutputParser() | search


In [32]:
chain.invoke({"input": "whats the name of the oldest cat?"})


'No good DuckDuckGo Search Result was found'